In [ ]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov  7 10:30:03 2020

@author: Administrator
"""

import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold



scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
drug = pd.read_csv('../input/lish-moa/train_drug.csv')


'''制作得分目标相关的'''
targets = scored.columns[1:]
scored = scored.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()


FOLDS = 7

for SEED in [42, 0, 1, 2, 3]:

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
              random_state=SEED)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)
        
        
    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
              random_state=SEED)
    tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)
    
    
    # ASSIGN FOLDS
    scored[f'fold_{SEED}'] = scored.drug_id.map(dct1)
    scored.loc[scored[f'fold_{SEED}'].isna(),[f'fold_{SEED}']] = scored.loc[scored[f'fold_{SEED}'].isna(),'sig_id'].map(dct2)
    scored[f'fold_{SEED}'] = scored[f'fold_{SEED}'].astype('int8')

del scored['drug_id']    
scored.to_csv('train_targets_scored.csv', index=False)



'''制作非得分目标相关的'''
scored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
drug = pd.read_csv('../input/lish-moa/train_drug.csv')


'''制作得分目标相关的'''
targets = scored.columns[1:]
scored = scored.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()


FOLDS = 7

for SEED in [42, 0, 1, 2, 3]:

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
              random_state=SEED)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)
        
        
    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
              random_state=SEED)
    tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)
    
    
    # ASSIGN FOLDS
    scored[f'fold_{SEED}'] = scored.drug_id.map(dct1)
    scored.loc[scored[f'fold_{SEED}'].isna(),[f'fold_{SEED}']] = scored.loc[scored[f'fold_{SEED}'].isna(),'sig_id'].map(dct2)
    scored[f'fold_{SEED}'] = scored[f'fold_{SEED}'].astype('int8')

del scored['drug_id']
scored.to_csv('train_targets_nonscored.csv', index=False)